In [1]:
import pandas as pd
import numpy as np
import matplotlib
from rdflib import Graph
from rdflib import Graph, Literal, Namespace, URIRef, BNode
from rdflib.namespace import RDF, RDFS, FOAF, OWL, XSD, DC, DCTERMS
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.request

def urify(ns, testo):
    testo = testo.replace(" ", "_").replace(".", "").replace(
        "/", "").replace("(", "").replace(")", "").replace("\'", "")
    return ns + urllib.parse.quote(testo)

def search_and_set_uri(uri_entity, property_name, resource):
    dbpedia_resource = urify(dbr, resource)

    if (None, None, URIRef(dbpedia_resource)) in g:
        g.add([URIRef(uri_entity), property_name, URIRef(dbpedia_resource)])
    else:

        sparql.setQuery("""
            ASK  {<""" + dbpedia_resource + """> ?p ?o }
        """)
        result = sparql.query().convert()

        if result['boolean'] is True:
            g.add([URIRef(uri_entity), property_name, URIRef(dbpedia_resource)])
        else:
            g.add([URIRef(uri_entity), property_name, Literal(str(resource))])


# Lettura del DataSet


In [2]:


path = 'C:\\Users\\claud\\OneDrive\\Desktop\\UNIVERSITA\\TERZO ANNO\\OPEN DATA\\Esame\\'
IncidentiSicilia = pd.read_csv(path + 'DatiConCadenzaSemestraleInfortuniSicilia.csv', sep =";")
IncidentiLazio = pd.read_csv(path + 'DatiConCadenzaSemestraleInfortuniLazio.csv', sep =";")
IncidentiLombardia = pd.read_csv(path + 'DatiConCadenzaSemestraleInfortuniLombardia.csv', sep =";")

#----------------------------------------------DataSetInfortuni--------------------------------------------------------------------------

mcp = pd.read_csv(path + 'Provincia.csv', sep = ";")

#-----------------------------------------------------------------------------------------------------------------------------------------

cn = pd.read_excel(path + 'CodiciNazioni.xls')
cn.to_csv('C:\\Users\\claud\\OneDrive\\Desktop\\UNIVERSITA\\TERZO ANNO\\OPEN DATA\\Esame\\cnInCSV.csv', index = False)
cnCSV = pd.read_csv (path + 'cnInCSV.csv')

#----------------------------------------Definizione Amministrativa-----------------------------------------------------------------

defamm = pd.read_csv(path + 'DefinizioneAmministrativa.csv', sep = ";")

#----------------------------------------TipologiaIndennizzo------------------------------------------------------------------------

tipindenn = pd.read_csv(path + 'TipologiaIndennizzo.csv', sep = ";")

#----------------------------------------DecisioneIstruttoriaEsitoMortale-----------------------------------------------------------

esitomortale = pd.read_csv(path + 'DecisioneIstruttoriaEsitoMortale.csv', sep= ";", encoding = 'latin-1')

#----------------------------------------LuogoNascita-------------------------------------------------------------------------

luogoNascita = pd.read_csv(path + 'LuogoNascita.csv', sep = ";")

# Pulizia del DataSet


In [3]:
def pulisci_dataset_infortuni(Dataset):
    
    Dataset = Dataset.drop(['DataRilevazione'], axis =1)
    Dataset = Dataset.drop(['DataProtocollo'], axis = 1)
    Dataset = Dataset.drop(['DataDefinizione'], axis = 1)
    #Dataset = Dataset.drop(['IdentificativoCaso'], axis = 1)
    Dataset = Dataset.drop(['IdentificativoInfortunato'], axis = 1)
    Dataset = Dataset.drop(['GradoMenomazione'], axis = 1)
    Dataset = Dataset.drop(['IdentificativoDatoreLavoro'], axis = 1)
    Dataset = Dataset.drop(['PosizioneAssicurativaTerritoriale'], axis = 1)
    Dataset = Dataset.drop(['Gestione'], axis = 1)
    Dataset = Dataset.drop(['GestioneTariffaria'], axis = 1)
    Dataset = Dataset.drop(['GrandeGruppoTariffario'], axis = 1)
    Dataset = Dataset.drop(['DefinizioneAmministrativaEsitoMortale'], axis = 1)
    Dataset = Dataset.drop(['SettoreAttivitaEconomica'], axis = 1)
    Dataset = Dataset.drop(['ModalitaAccadimento'], axis = 1)

    dataset = pd.concat([Dataset], axis=1)
   
    return dataset

In [4]:
InfortuniSicilia_final = pulisci_dataset_infortuni(IncidentiSicilia)
InfortuniLazio_final = pulisci_dataset_infortuni(IncidentiLazio)
InfortuniLombardia_final = pulisci_dataset_infortuni(IncidentiLombardia)

In [5]:
final_merge = pd.concat([
    InfortuniSicilia_final,
    InfortuniLazio_final,
    InfortuniLombardia_final,

], ignore_index = True)

In [6]:
final_merge['DataAccadimento'] = pd.to_datetime(final_merge['DataAccadimento'], format='%d/%m/%Y')


# Ridefinisci il formato della colonna "DataAccadimento" come "aaaa"
final_merge['DataAccadimento'] = final_merge['DataAccadimento'].dt.strftime('%Y')

final_merge = final_merge.sort_values(['LuogoAccadimento', 'DataAccadimento'])

In [7]:
mcp = mcp.drop(["CodCittaMetropolitana", "CodRegione", "CodMacroregione", "DescrMacroregione", "CodNazione", "DescrNazione", "DataInizioValidita", "DataFineValidita" ],  axis=1)
mcp = mcp.drop(mcp[(mcp.DescrRegione != "Sicilia") & (mcp.DescrRegione != "Lombardia") & (mcp.DescrRegione != "Lazio")].index, axis = 0)

In [8]:
dataset_fuso = pd.merge(final_merge, mcp, left_on='LuogoAccadimento', right_on='Provincia', how='left')
datasetfuso1 = pd.merge(final_merge, defamm, left_on='DefinizioneAmministrativa', right_on='DefinizioneAmministrativa', how='left')
datasetfuso2 = pd.merge(final_merge, tipindenn, left_on='Indennizzo', right_on='TipologiaIndennizzo', how='left')
datasetfuso3 = pd.merge(final_merge, esitomortale, left_on='DecisioneIstruttoriaEsitoMortale', right_on='DecisioneIstruttoriaEsitoMortale', how='left')
datasetfuso4 = pd.merge(final_merge, luogoNascita, left_on='LuogoNascita', right_on='LuogoNascita', how='left')

final_merge['LuogoAccadimento'] = dataset_fuso['DescrProvincia']
final_merge['DefinizioneAmministrativa'] = datasetfuso1['DescrDefinizioneAmministrativa']
final_merge['Indennizzo'] = datasetfuso2['DescrTipologiaIndennizzo']
final_merge['DecisioneIstruttoriaEsitoMortale'] = datasetfuso3['DescrDecisioneIstruttoriaEsitoMortale']
final_merge['LuogoNascita'] = datasetfuso4['DescrNazioneNascita']

In [9]:
final_merge.to_csv('C:\\Users\\claud\\OneDrive\\Desktop\\UNIVERSITA\\TERZO ANNO\\OPEN DATA\\Esame\\Infortuni_final.csv', index=False)
final_merge = final_merge.reset_index(drop=True)

final_merge

,DataAccadimento,DataMorte,LuogoAccadimento,Genere,Eta,LuogoNascita,ConSenzaMezzoTrasporto,IdentificativoCaso,DefinizioneAmministrativa,Indennizzo,DecisioneIstruttoriaEsitoMortale,GiorniIndennizzati
0,2017,NaN,Bergamo,M,55,ROMANIA,N,21073040,Positivo,In temporanea,Non Applicabile,2
1,2017,NaN,Bergamo,F,53,ITALIA,N,20698071,Positivo,In temporanea,Non Applicabile,0
2,2017,NaN,Bergamo,M,47,ITALIA,S,20711968,Negativo,Nessuno,Non Applicabile,0
3,2017,NaN,Bergamo,M,42,ITALIA,N,20624771,Positivo,In temporanea,Non Applicabile,152
4,2017,NaN,Bergamo,F,43,ITALIA,N,21032885,Positivo,In temporanea,Non Applicabile,76
...,...,...,...,...,...,...,...,...,...,...,...,...
924023,2021,NaN,Monza E Della Brianza,F,48,ITALIA,S,23714961,Positivo,In temporanea,Non Applicabile,19
924024,2021,NaN,Monza E Della Brianza,M,48,ITALIA,S,23860777,Positivo,Nessuno,Non Applicabile,22
924025,2021,NaN,Monza E Della Brianza,M,20,ITALIA,N,23666452,Negativo,Nessuno,Non Applicabile,5
924026,2021,NaN,Monza E Della Brianza,M,55,ITALIA,N,23461398,Positivo,In temporanea,Non Applicabile,0


# Ontologia


In [10]:
# Creazione del grafo RDF
g = Graph()

uri_base = "http://www.IncidentiSulLavoro.it/resource/"
inc = "http://www.IncidentiSulLavoro.it/resource/incidenti"
inlo = Namespace("http://www.IncidentiSulLavoro.it/ontology/")
dbr = Namespace("http://www.dbpedia.org/resource/")

#-------------------------------------------------

g.add((inlo.Persona, RDF.type, RDFS.Class))

g.add((inlo.Genere, RDF.type, RDF.Property))
g.add((inlo.Genere, RDFS.domain, inlo.Persona))
g.add((inlo.Genere, RDFS.range, RDFS.Literal))

g.add((inlo.Eta, RDF.type, RDF.Property))
g.add((inlo.Eta, RDFS.domain, inlo.Persona))
g.add((inlo.Eta, RDFS.range, XSD.integer))

g.add((inlo.LuogoNascita, RDF.type, RDF.Property))
g.add((inlo.LuogoNascita, RDFS.domain, inlo.Persona))
g.add((inlo.LuogoNascita, RDFS.range, RDFS.Literal))

#-----------------------------------------------------------------------------

g.add((inlo.Luogo, RDF.type, RDFS.Class))

g.add((inlo.LuogoAccadimento, RDF.type, RDF.Property))
g.add((inlo.LuogoAccadimento, RDFS.domain, inlo.Luogo))
g.add((inlo.LuogoAccadimento, RDFS.range, RDFS.Literal))

#-----------------------------------------------------------------------------

g.add((inlo.IncidentiSulLavoro, RDF.type, RDFS.Class))

g.add((inlo.DataAccadimento, RDF.type, RDF.Property))
g.add((inlo.DataAccadimento, RDFS.domain, inlo.IncidentiSulLavro))
g.add((inlo.DataAccadimento, RDFS.range, XSD.date))

g.add((inlo.DataMorte, RDF.type, RDF.Property))
g.add((inlo.DataMorte, RDFS.domain, inlo.IncidentiSulLavoro))
g.add((inlo.DataMorte, RDFS.range, XSD.date))

g.add((inlo.ConSenzaMezzoTrasporto, RDF.type, RDF.Property))
g.add((inlo.ConSenzaMezzoTrasporto, RDFS.domain, inlo.IncidentiSulLavoro))
g.add((inlo.ConSenzaMezzoTrasporto, RDFS.range, RDFS.Literal))

g.add((inlo.DefinizioneAmministrativa, RDF.type, RDF.Property))
g.add((inlo.DefinizioneAmministrativa, RDFS.domain, inlo.IncidentiSulLavoro))
g.add((inlo.DefinizioneAmministrativa, RDFS.range, RDFS.Literal))

g.add((inlo.Indennizzo, RDF.type, RDF.Property))
g.add((inlo.Indennizzo, RDFS.domain, inlo.IncidentiSulLavoro))
g.add((inlo.Indennizzo, RDFS.range, RDFS.Literal))

g.add((inlo.DecisioneIstruttoriaEsitoMortale, RDF.type, RDF.Property))
g.add((inlo.DecisioneIstruttoriaEsitoMortale, RDFS.domain, inlo.IncidentiSulLavoro))
g.add((inlo.DecisioneIstruttoriaEsitoMortale, RDFS.range, RDFS.Literal))

g.add((inlo.GiorniIndennizzati, RDF.type, RDF.Property))
g.add((inlo.GiorniIndennizzati, RDFS.domain, inlo.IncidentiSulLavoro))
g.add((inlo.GiorniIndennizzati, RDFS.range, XSD.integer))

g.add((inlo.persona, RDF.type, RDF.Property))
g.add((inlo.persona, RDFS.domain, inlo.IncidentiSulLavoro))
g.add((inlo.persona, RDFS.range, inlo.Persona))

g.add((inlo.luogo, RDF.type, RDF.Property))
g.add((inlo.luogo, RDFS.domain, inlo.IncidentiSulLavoro))
g.add((inlo.luogo, RDFS.range, inlo.Luogo))

g.add((inlo.Identificativocaso, RDF.type, RDF.Property))
g.add((inlo.Identificativocaso, RDFS.domain, inlo.IncidentiSulLavoro))
g.add((inlo.Identificativocaso, RDFS.range, XSD.integer))

#--------------------------------------------------------------------------------

g.serialize(destination='ontologia.ttl', format='turtle')

<Graph identifier=N22dd5fc9511547db8a8071cdfc815139 (<class 'rdflib.graph.Graph'>)>

In [11]:
sparql = SPARQLWrapper('http://dbpedia.ord/sparql')
sparql.setReturnFormat(JSON)

lista_citta = []
for place in final_merge["LuogoAccadimento"]:
    lista_citta.append(place)

my_set = {s for s in lista_citta}


for place in my_set:
    bn = BNode()

    ape = urify(uri_base, place)
    print(place)
    g.add((URIRef(ape), RDF.type, URIRef(inlo + "Luogo")))
    g.add((URIRef(ape), URIRef(inlo + "Has_Incident"), bn))

    for incident in final_merge["IdentificativoCaso"]:
        g.add((URIRef(uri_base + str(incident)), RDF.type, URIRef(inlo + "IncidentiLavorativi")))
        g.add((bn, URIRef(inlo + "Has_ID"), URIRef(uri_base + str(incident))))
        

g.serialize(destination='risultato_ontologia_coll.ttl', format='turtle')

Agrigento
Caltanissetta
Brescia
Catania
Palermo
Como
Roma
Latina
Enna
Mantova
Frosinone
Messina
Siracusa
Lecco
Monza E Della Brianza
Pavia
Rieti
